In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import copy

from tqdm import tqdm
from scipy.spatial import ConvexHull

from util import load_data, separate_data
from models.graphcnn import GraphCNN

In [91]:
args = {
    'dataset': 'MUTAG',
    'device': 0,
    'batch_size': 32,
    'iters_per_epoch': 50,
    'epochs': 5,
    'lr': 0.01,
    'seed': 0,
    'fold_idx': 0,
    'num_layers': 5,
    'num_mlp_layers': 2,
    'hidden_dim': 3,
    'final_dropout': 0.5,
    'graph_pooling_type': 'sum',
    'neighbor_pooling_type': 'sum',
    'learn_eps': False,
    'degree_as_tag': False,
    'filename': ""
}

In [92]:
def train(args, model, device, train_graphs, optimizer, epoch):
    model.train()

    total_iters = args['iters_per_epoch']
    pbar = tqdm(range(total_iters), unit='batch')

    loss_accum = 0
    for pos in pbar:
        selected_idx = np.random.permutation(len(train_graphs))[:args['batch_size']]

        batch_graph = [train_graphs[idx] for idx in selected_idx]
        output = model(batch_graph)

        labels = torch.LongTensor([graph.label for graph in batch_graph]).to(device)

        #compute loss
        loss = criterion(output, labels)

        #backprop
        if optimizer is not None:
            optimizer.zero_grad()
            loss.backward()         
            optimizer.step()
        

        loss = loss.detach().cpu().numpy()
        loss_accum += loss

        #report
        pbar.set_description('epoch: %d' % (epoch))

    average_loss = loss_accum/total_iters
    print("loss training: %f" % (average_loss))
    
    return average_loss

###pass data to model with minibatch during testing to avoid memory overflow (does not perform backpropagation)
def pass_data_iteratively(model, graphs, minibatch_size = 64):
    model.eval()
    output = []
    idx = np.arange(len(graphs))
    for i in range(0, len(graphs), minibatch_size):
        sampled_idx = idx[i:i+minibatch_size]
        if len(sampled_idx) == 0:
            continue
        output.append(model([graphs[j] for j in sampled_idx]).detach())
    return torch.cat(output, 0)

def test(args, model, device, train_graphs, test_graphs, epoch):
    model.eval()

    output = pass_data_iteratively(model, train_graphs)
    pred = output.max(1, keepdim=True)[1]
    labels = torch.LongTensor([graph.label for graph in train_graphs]).to(device)
    correct = pred.eq(labels.view_as(pred)).sum().cpu().item()
    acc_train = correct / float(len(train_graphs))

    output = pass_data_iteratively(model, test_graphs)
    pred = output.max(1, keepdim=True)[1]
    labels = torch.LongTensor([graph.label for graph in test_graphs]).to(device)
    correct = pred.eq(labels.view_as(pred)).sum().cpu().item()
    acc_test = correct / float(len(test_graphs))

    print("accuracy train: %f test: %f" % (acc_train, acc_test))

    return acc_train, acc_test

In [93]:
class FeatsGraphCNN(GraphCNN):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def __preprocess_neighbors_maxpool(self, batch_graph):
        ###create padded_neighbor_list in concatenated graph

        #compute the maximum number of neighbors within the graphs in the current minibatch
        max_deg = max([graph.max_neighbor for graph in batch_graph])

        padded_neighbor_list = []
        start_idx = [0]


        for i, graph in enumerate(batch_graph):
            start_idx.append(start_idx[i] + len(graph.g))
            padded_neighbors = []
            for j in range(len(graph.neighbors)):
                #add off-set values to the neighbor indices
                pad = [n + start_idx[i] for n in graph.neighbors[j]]
                #padding, dummy data is assumed to be stored in -1
                pad.extend([-1]*(max_deg - len(pad)))

                #Add center nodes in the maxpooling if learn_eps is False, i.e., aggregate center nodes and neighbor nodes altogether.
                if not self.learn_eps:
                    pad.append(j + start_idx[i])

                padded_neighbors.append(pad)
            padded_neighbor_list.extend(padded_neighbors)

        return torch.LongTensor(padded_neighbor_list)


    def __preprocess_neighbors_sumavepool(self, batch_graph):
        ###create block diagonal sparse matrix

        edge_mat_list = []
        start_idx = [0]
        for i, graph in enumerate(batch_graph):
            start_idx.append(start_idx[i] + len(graph.g))
            edge_mat_list.append(graph.edge_mat + start_idx[i])
        Adj_block_idx = torch.cat(edge_mat_list, 1)
        Adj_block_elem = torch.ones(Adj_block_idx.shape[1])

        #Add self-loops in the adjacency matrix if learn_eps is False, i.e., aggregate center nodes and neighbor nodes altogether.

        if not self.learn_eps:
            num_node = start_idx[-1]
            self_loop_edge = torch.LongTensor([range(num_node), range(num_node)])
            elem = torch.ones(num_node)
            Adj_block_idx = torch.cat([Adj_block_idx, self_loop_edge], 1)
            Adj_block_elem = torch.cat([Adj_block_elem, elem], 0)

        Adj_block = torch.sparse.FloatTensor(Adj_block_idx, Adj_block_elem, torch.Size([start_idx[-1],start_idx[-1]]))

        return Adj_block.to(self.device)


    def __preprocess_graphpool(self, batch_graph):
        ###create sum or average pooling sparse matrix over entire nodes in each graph (num graphs x num nodes)
        
        start_idx = [0]

        #compute the padded neighbor list
        for i, graph in enumerate(batch_graph):
            start_idx.append(start_idx[i] + len(graph.g))

        idx = []
        elem = []
        for i, graph in enumerate(batch_graph):
            ###average pooling
            if self.graph_pooling_type == "average":
                elem.extend([1./len(graph.g)]*len(graph.g))
            
            else:
            ###sum pooling
                elem.extend([1]*len(graph.g))

            idx.extend([[i, j] for j in range(start_idx[i], start_idx[i+1], 1)])
        elem = torch.FloatTensor(elem)
        idx = torch.LongTensor(idx).transpose(0,1)
        graph_pool = torch.sparse.FloatTensor(idx, elem, torch.Size([len(batch_graph), start_idx[-1]]))
        
        return graph_pool.to(self.device)
    
    def features(self, batch_graph):
        X_concat = torch.cat([graph.node_features for graph in batch_graph], 0).to(self.device)
        graph_pool = self.__preprocess_graphpool(batch_graph)

        if self.neighbor_pooling_type == "max":
            padded_neighbor_list = self.__preprocess_neighbors_maxpool(batch_graph)
        else:
            Adj_block = self.__preprocess_neighbors_sumavepool(batch_graph)

        #list of hidden representation at each layer (including input)
        hidden_rep = [X_concat]
        h = X_concat

        for layer in range(self.num_layers-1):
            if self.neighbor_pooling_type == "max" and self.learn_eps:
                h = self.next_layer_eps(h, layer, padded_neighbor_list = padded_neighbor_list)
            elif not self.neighbor_pooling_type == "max" and self.learn_eps:
                h = self.next_layer_eps(h, layer, Adj_block = Adj_block)
            elif self.neighbor_pooling_type == "max" and not self.learn_eps:
                h = self.next_layer(h, layer, padded_neighbor_list = padded_neighbor_list)
            elif not self.neighbor_pooling_type == "max" and not self.learn_eps:
                h = self.next_layer(h, layer, Adj_block = Adj_block)

            hidden_rep.append(h)
        
        pooleds = [graph_pool]
        #perform pooling over all nodes in each graph in every layer
        for layer, h in enumerate(hidden_rep):
            pooled_h = torch.spmm(graph_pool, h)
            pooleds.append(pooled_h)
            
        return hidden_rep, pooleds

In [94]:
criterion = nn.CrossEntropyLoss()

#set up seeds and gpu device
torch.manual_seed(0)
np.random.seed(0)    
device = torch.device("cuda:" + str(args['device'])) if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)

graphs, num_classes = load_data(args['dataset'], args['degree_as_tag'])

##10-fold cross validation. Conduct an experiment on the fold specified by args.fold_idx.
train_graphs, test_graphs = separate_data(graphs, args['seed'], args['fold_idx'])

model = FeatsGraphCNN(args['num_layers'], args['num_mlp_layers'], train_graphs[0].node_features.shape[1], args['hidden_dim'], num_classes, args['final_dropout'], args['learn_eps'], args['graph_pooling_type'], args['neighbor_pooling_type'], device).to(device)

optimizer = optim.Adam(model.parameters(), lr=args['lr'])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

loading data
# classes: 2
# maximum node tag: 7
# data: 188


In [95]:
for epoch in range(1, args['epochs'] + 1):
    scheduler.step()

    avg_loss = train(args, model, device, train_graphs, optimizer, epoch)
    acc_train, acc_test = test(args, model, device, train_graphs, test_graphs, epoch)

    if not args['filename'] == "":
        with open(args['filename'], 'w') as f:
            f.write("%f %f %f" % (avg_loss, acc_train, acc_test))
            f.write("\n")
    print("")

    print(model.eps)

/home/wgar/anaconda3/envs/geo-const/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
epoch: 2:   6%|▌         | 3/50 [00:00<00:01, 26.10batch/s]

loss training: 1.775890
accuracy train: 0.761905 test: 0.750000

Parameter containing:
tensor([0., 0., 0., 0.], device='cuda:0', requires_grad=True)


epoch: 3:   6%|▌         | 3/50 [00:00<00:01, 26.49batch/s]

loss training: 0.447053
accuracy train: 0.821429 test: 0.800000

Parameter containing:
tensor([0., 0., 0., 0.], device='cuda:0', requires_grad=True)


epoch: 4:   6%|▌         | 3/50 [00:00<00:01, 24.86batch/s]

loss training: 0.375520
accuracy train: 0.839286 test: 0.800000

Parameter containing:
tensor([0., 0., 0., 0.], device='cuda:0', requires_grad=True)


epoch: 5:   6%|▌         | 3/50 [00:00<00:01, 25.09batch/s]

loss training: 0.356932
accuracy train: 0.857143 test: 0.750000

Parameter containing:
tensor([0., 0., 0., 0.], device='cuda:0', requires_grad=True)


epoch: 5: 100%|██████████| 50/50 [00:01<00:00, 25.60batch/s]


loss training: 0.348208
accuracy train: 0.827381 test: 0.800000

Parameter containing:
tensor([0., 0., 0., 0.], device='cuda:0', requires_grad=True)


In [66]:
selected_idx = np.random.permutation(len(train_graphs))[:args['batch_size']]

batch_graph = [train_graphs[idx] for idx in selected_idx]
hiddens, pooleds = model.features(batch_graph)

In [67]:
for h in hiddens:
    print(h.shape)

torch.Size([607, 7])
torch.Size([607, 3])
torch.Size([607, 3])
torch.Size([607, 3])
torch.Size([607, 3])


In [68]:
for p in pooleds:
    print(p.shape)

torch.Size([32, 607])
torch.Size([32, 7])
torch.Size([32, 3])
torch.Size([32, 3])
torch.Size([32, 3])
torch.Size([32, 3])


In [69]:
v = pooleds[2]
v.shape

torch.Size([32, 3])

In [12]:
n, h = v.shape

In [13]:
v.unsqueeze(2).repeat((1, 1, h)).shape

torch.Size([32, 8, 8])

In [14]:
inter_feature_distance_matrix = (v.unsqueeze(2).repeat((1, 1, h)) - v.unsqueeze(1)).abs()

In [15]:
 (v.unsqueeze(0).repeat((n, 1, 1)) - v.unsqueeze(1)).sum(dim=2).abs()[:4, :4]

tensor([[ 0.0000, 23.7763, 27.4454, 16.6963],
        [23.7763,  0.0000,  3.6690,  7.0801],
        [27.4454,  3.6690,  0.0000, 10.7491],
        [16.6963,  7.0801, 10.7491,  0.0000]], device='cuda:0',
       grad_fn=<SliceBackward>)

In [16]:
torch.sum(v[0] - v[0]), torch.sum(v[0] - v[1]), torch.sum(v[0] - v[2]), torch.sum(v[0] - v[2])

(tensor(0., device='cuda:0', grad_fn=<SumBackward0>),
 tensor(23.7763, device='cuda:0', grad_fn=<SumBackward0>),
 tensor(27.4454, device='cuda:0', grad_fn=<SumBackward0>),
 tensor(27.4454, device='cuda:0', grad_fn=<SumBackward0>))

In [17]:
inter_batch_distance_matrix =  (v.unsqueeze(0).repeat((n, 1, 1)) - v.unsqueeze(1)).sum(dim=2).abs()

In [18]:
dist_stacked = torch.cat([inter_batch_distance_matrix ** 2, torch.ones((1, n)).cuda()], dim=0)

In [19]:
dist_stacked = torch.cat([dist_stacked, torch.ones((n+1, 1)).cuda()], dim=1)

In [20]:
dist_stacked[-4:, -4:]

tensor([[  0.0000,  93.8940, 417.7198,   1.0000],
        [ 93.8940,   0.0000, 115.5260,   1.0000],
        [417.7198, 115.5260,   0.0000,   1.0000],
        [  1.0000,   1.0000,   1.0000,   1.0000]], device='cuda:0',
       grad_fn=<SliceBackward>)

In [21]:
dist_stacked[n, n] = 0.

In [22]:
dist_stacked[-4:, -4:]

tensor([[  0.0000,  93.8940, 417.7198,   1.0000],
        [ 93.8940,   0.0000, 115.5260,   1.0000],
        [417.7198, 115.5260,   0.0000,   1.0000],
        [  1.0000,   1.0000,   1.0000,   0.0000]], device='cuda:0',
       grad_fn=<SliceBackward>)

In [23]:
CM = dist_stacked.det()

In [24]:
CM

tensor(0., device='cuda:0', grad_fn=<DetBackward>)

In [25]:
simplex_volume = (-1 ** (n + 1)) / ((np.math.factorial(n) ** 2) * (2 ** n)) * CM

In [26]:
simplex_volume

tensor(-0., device='cuda:0', grad_fn=<MulBackward0>)

In [27]:
def inter_feature_distances(M: torch.tensor):
    n, h = M.shape
    return (M.unsqueeze(2).repeat((1, 1, h)) - M.unsqueeze(1)) ** 2

In [28]:
def inter_batch_distances(M: torch.tensor):
    n, h = M.shape
    return (M.unsqueeze(0).repeat((n, 1, 1)) - M.unsqueeze(1)).sum(dim=2) ** 2

In [108]:
def inter_batch_distance_vec(M: torch.tensor):
    n, h = M.shape
    return (M.unsqueeze(0).repeat((n, 1, 1)) - M.unsqueeze(1)) ** 2

In [29]:
def cayley_menger_determinant(D: torch.tensor):
    n = D.shape[0]
    dist_stacked = torch.cat([D, torch.ones((1, n)).cuda()], dim=0)
    dist_stacked = torch.cat([dist_stacked, torch.ones((n+1, 1)).cuda()], dim=1)
    dist_stacked[n, n] = 0.
    CM = dist_stacked.det()
    return CM

In [30]:
def simplex_volume(n: int, CM: torch.tensor):
    return (-1 ** (n + 1)) / ((np.math.factorial(n) ** 2) * (2 ** n)) * CM

In [96]:
n = 24
print(n)
selected_idx = np.random.permutation(len(train_graphs))[:n]

batch_graph = [train_graphs[idx] for idx in selected_idx]
labels = torch.LongTensor([graph.label for graph in batch_graph]).to(device)

len_a = len((labels == 0).nonzero().squeeze(1).detach().cpu().numpy()) > 0
len_b = len((labels == 1).nonzero().squeeze(1).detach().cpu().numpy()) > 0
class_a_dict = {'graph': [batch_graph[idx] for idx in (labels == 0).nonzero().squeeze(1)], 'name': '0'} if len_a else None
class_b_dict = {'graph': [batch_graph[idx] for idx in (labels == 1).nonzero().squeeze(1)], 'name': '1'} if len_b else None

for class_dict in (class_a_dict, class_b_dict):
    if class_dict is None:
        continue
    print(class_dict['name'])
    hiddens, pooleds = model.features(class_dict['graph'])
    n = len(class_dict['graph'])
    for pooled in pooleds[1:]:
#         print(simplex_volume(n, cayley_menger_determinant(inter_batch_distances(pooled))))
        print(cayley_menger_determinant(inter_batch_distances(pooled)))

24
0
tensor(0., device='cuda:0')
tensor(-0., device='cuda:0', grad_fn=<DetBackward>)
tensor(-0., device='cuda:0', grad_fn=<DetBackward>)
tensor(-0., device='cuda:0', grad_fn=<DetBackward>)
tensor(0., device='cuda:0', grad_fn=<DetBackward>)
1
tensor(-0., device='cuda:0')
tensor(-4.5550e-28, device='cuda:0', grad_fn=<DetBackward>)
tensor(7.4686e-29, device='cuda:0', grad_fn=<DetBackward>)
tensor(0., device='cuda:0', grad_fn=<DetBackward>)
tensor(-5.3351e-37, device='cuda:0', grad_fn=<DetBackward>)


In [107]:
for class_dict in (class_a_dict, class_b_dict):
    if class_dict is None:
        continue
    print(class_dict['name'])
    hiddens, pooleds = model.features(class_dict['graph'])
    n = len(class_dict['graph'])
    print(n)
    pooled = pooleds[-1].detach().cpu().numpy()
    print(pooled)
    if class_dict['name'] is 0:
        hull = ConvexHull(pooled[:4])
        print(hull.volume)
    else:
        hull = ConvexHull(pooled[:3, 1:])
        print(hull.area)
    

0
17
[[20.224258    2.2084515   0.8439397 ]
 [ 5.9901342   1.2501658   0.78397995]
 [ 3.3844457   1.1016296   1.0156009 ]
 [ 1.3948257   1.5147442   0.8645391 ]
 [ 0.          3.0422933   0.12203759]
 [ 0.          1.5158514   0.45665187]
 [14.762299    1.1016296   1.1948199 ]
 [ 9.257972    2.2032592   1.2259839 ]
 [ 1.3948257   1.5147442   0.7976161 ]
 [14.350725    1.1068219   0.63355666]
 [ 1.4514918   1.5147442   0.8150811 ]
 [ 0.          1.5161711   0.72425836]
 [13.294289    1.1016296   1.3076241 ]
 [ 1.4514918   1.5147442   0.8820039 ]
 [ 9.258654    2.2026293   1.2262492 ]
 [ 0.          3.0317028   0.6456123 ]
 [ 1.3948257   1.5147442   0.9983848 ]]
2.355371100388537
1
7
[[0.         1.5158514  0.45665187]
 [0.         1.5161711  0.6573355 ]
 [0.         1.5161711  0.38964415]
 [0.         1.5556002  1.1170869 ]
 [0.         1.5161711  0.9250269 ]
 [0.         4.01882    0.51583576]
 [0.         2.178091   0.40153712]]
0.5353837650773908


In [468]:
(labels == 0).nonzero().squeeze(1).shape

torch.Size([7])

In [928]:
n = 1
print(n)
selected_idx = np.random.permutation(len(train_graphs))[:n]
selected_idx_different = np.random.permutation(len(train_graphs))[:n]

batch_graph = [train_graphs[idx] for idx in selected_idx]
batch_graph2 = copy.deepcopy(batch_graph)
for i in range(len(batch_graph2)):
    batch_graph2[i].node_features[:, 0] += 0.2

batch_graph3 = [train_graphs[idx] for idx in selected_idx_different]
    
print(class_b_dict['name'])
hiddens, pooleds = model.features(batch_graph)
hiddens2, pooleds2 = model.features(batch_graph2)
hiddens3, pooleds3 = model.features(batch_graph3)

n = len(class_b_dict['graph'])
# for pooled in pooleds[1:]:
#         print(simplex_volume(n, cayley_menger_determinant(inter_batch_distances(pooled))))
# print(cayley_menger_determinant(inter_batch_distances(pooleds[-1])))
# print(cayley_menger_determinant(inter_batch_distances(pooleds[-1])))
# print(cayley_menger_determinant(inter_batch_distances(pooleds2[-1])))

print(inter_feature_distances(pooleds[-1]))
print(inter_feature_distances(pooleds2[-1]))
print((pooleds[-1] - pooleds2[-1]).norm())
print((pooleds[-1] - pooleds3[-1]).norm())

batch_all = np.concatenate([batch_graph, batch_graph2], axis=0)
output = pass_data_iteratively(model, batch_all)
pred = output.max(1, keepdim=True)[1]
labels = torch.LongTensor([graph.label for graph in batch_all]).to(device)
correct = pred.eq(labels.view_as(pred)).cpu()
print(correct)

1
1
tensor([[[ 0.0000,  0.8859,  1.0149,  ..., 10.9658, 10.4178,  0.8206],
         [ 0.8859,  0.0000,  1.9008,  ..., 11.8517, 11.3037,  0.0653],
         [ 1.0149,  1.9008,  0.0000,  ...,  9.9509,  9.4029,  1.8356],
         ...,
         [10.9658, 11.8517,  9.9509,  ...,  0.0000,  0.5480, 11.7865],
         [10.4178, 11.3037,  9.4029,  ...,  0.5480,  0.0000, 11.2385],
         [ 0.8206,  0.0653,  1.8356,  ..., 11.7865, 11.2385,  0.0000]]],
       device='cuda:0', grad_fn=<AbsBackward>)
tensor([[[0.0000e+00, 1.4890e+00, 7.9843e-02,  ..., 6.2033e+01,
          8.5789e+01, 0.0000e+00],
         [1.4890e+00, 0.0000e+00, 1.4092e+00,  ..., 6.0544e+01,
          8.4300e+01, 1.4890e+00],
         [7.9843e-02, 1.4092e+00, 0.0000e+00,  ..., 6.1953e+01,
          8.5709e+01, 7.9843e-02],
         ...,
         [6.2033e+01, 6.0544e+01, 6.1953e+01,  ..., 0.0000e+00,
          2.3756e+01, 6.2033e+01],
         [8.5789e+01, 8.4300e+01, 8.5709e+01,  ..., 2.3756e+01,
          0.0000e+00, 8.5789e+01]

In [929]:
batch_graph2[0].node_features

tensor([[1.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.2000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1

In [930]:
batch_graph[0].node_features

tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.]])